In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.6.0+cu118


In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
query = inputs [1]

attn_score_2 = torch.empty((inputs.shape[0]))
for i , x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(query, x_i)

print("attn_score_2:", attn_score_2)

attn_score_2: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
def softmax_naive(x):
    return torch.exp(x) / torch.sum(torch.exp(x), dim=0)

attn_score_2_softmax = softmax_naive(attn_score_2)

print("attn_score_2_softmax:", attn_score_2_softmax)
print("sum(attn_score_2_softmax):", torch.sum(attn_score_2_softmax))   

attn_score_2_softmax: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum(attn_score_2_softmax): tensor(1.)


In [5]:
attn_weights_2 = torch.softmax(attn_score_2, dim=0)


In [6]:
query = inputs [1]
context_vec_2 = torch.zeros(query.shape)
for i , x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print("context_vec_2:", context_vec_2)

context_vec_2: tensor([0.4419, 0.6515, 0.5683])


In [7]:
attn_scores = inputs @ inputs.T
print("attn_scores:", attn_scores)

attn_scores: tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [8]:
attn_weights = torch.softmax(attn_scores, dim=1)
print("attn_weights:", attn_weights)

attn_weights: tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [9]:
all_context_vecs = attn_weights @ inputs
print("all_context_vecs:", all_context_vecs)

all_context_vecs: tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [10]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2    

In [13]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)

In [14]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print("query_2:", query_2)

query_2: tensor([-1.1729, -0.0048])


In [15]:
keys = inputs @ W_key
values = inputs @ W_value



In [16]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print("attn_score_22:", attn_score_22)

attn_score_22: tensor(0.1376)


In [19]:
attn_scores_2 = query_2 @ keys.T
print("attn_score_2:", attn_score_2)

attn_score_2: tensor([ 0.2172,  0.1376,  0.1730, -0.0491,  0.7616, -0.3809])


In [22]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1704, 0.1611, 0.1652, 0.1412, 0.2505, 0.1117])


In [23]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.2854, 0.4081])


## 3.4

In [ ]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.randn(d_in, d_out))
        self.W_key = nn.Parameter(torch.randn(d_in, d_out))
        self.W_value = nn.Parameter(torch.randn(d_in, d_out))


    def forward(self, x):
        keys = x @ self.W_key
        values = x @ self.W_value
        queries = x @ self.W_query

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec
    
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2845, 0.4071],
        [0.2854, 0.4081],
        [0.2854, 0.4075],
        [0.2864, 0.3974],
        [0.2863, 0.3910],
        [0.2860, 0.4039]], grad_fn=<MmBackward0>)


In [25]:
class selfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out)
        self.W_key = nn.Linear(d_in, d_out)
        self.W_value = nn.Linear(d_in, d_out)

    def forward(self, x):
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec
    
torch.manual_seed(123)
sa_v2 = selfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[0.1059, 0.9296],
        [0.1144, 0.9353],
        [0.1143, 0.9353],
        [0.1181, 0.9369],
        [0.1138, 0.9343],
        [0.1188, 0.9375]], grad_fn=<MmBackward0>)


## 3.5

In [26]:
batch = torch.stack([inputs, inputs], dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [35]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out,
                 context_legnth, dropout, qkv_bias=False):
            
            super().__init__()
            self.d_out = d_out
            self.W_query = nn.Linear(d_in, d_out)
            self.W_key = nn.Linear(d_in, d_out)
            self.W_value = nn.Linear(d_in, d_out)
            self.dropout = nn.Dropout(dropout)
            self.register_buffer("mask", torch.tril(torch.ones(context_legnth, context_legnth), diagonal=1))

    def forward(self, x):
          
          b, num_tokens, d_in = x.shape
        
          keys = self.W_key(x)
          values = self.W_value(x)
          queries = self.W_query(x)

          attn_scores = queries @ keys.transpose(1, 2)
          attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
          attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
          attn_weights = self.dropout(attn_weights)

          context_vec = attn_weights @ values
          return context_vec
    
torch.manual_seed(123)

context_legnth = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_legnth, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print(context_vecs.shape)

tensor([[[0.1505, 0.9278],
         [0.1690, 0.8795],
         [0.1362, 0.8840],
         [0.3634, 0.9523],
         [   nan,    nan],
         [   nan,    nan]],

        [[0.1505, 0.9278],
         [0.1690, 0.8795],
         [0.1362, 0.8840],
         [0.3634, 0.9523],
         [   nan,    nan],
         [   nan,    nan]]], grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 6, 2])


In [36]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2, qkv_bias = False)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 0.1505,  0.9278, -0.3476,  0.2902],
         [ 0.1690,  0.8795, -0.3114,  0.2670],
         [ 0.1362,  0.8840, -0.3117,  0.3048],
         [ 0.3634,  0.9523, -0.3721,  0.0918],
         [    nan,     nan,     nan,     nan],
         [    nan,     nan,     nan,     nan]],

        [[ 0.1505,  0.9278, -0.3476,  0.2902],
         [ 0.1690,  0.8795, -0.3114,  0.2670],
         [ 0.1362,  0.8840, -0.3117,  0.3048],
         [ 0.3634,  0.9523, -0.3721,  0.0918],
         [    nan,     nan,     nan,     nan],
         [    nan,     nan,     nan,     nan]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [37]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forwar

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
